In [15]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_2/execute_code.jsonl")
data_2 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_3/execute_code.jsonl")
data_3 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_4/execute_code.jsonl")

data_sft = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/sft-data/gemini2.5/3.5k.json"))

data_evo = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/sft-data/sft_industry_optmath_evo_more.json"))

for d in data_evo:
    # t = {'prompt': question2prompt(d['en_question']), 'completion': d['en_math_model_coptpy_code']}
    import re

    match = re.search(r"Question:\s*([\s\S]+)$", d['prompt'])
    
    question = match.group(1).strip()
        
    d['question'] = question

In [16]:
data_im1 = data_1 + data_2 

data_im2 = data_sft + data_evo+ data_3

In [33]:
len(data_im2)

15105

In [34]:
def is_numeric_str(s):
    # 如果是 None (通常是 .get() 没找到 key 的情况)，在这里我们单独在外面逻辑判断
    if not isinstance(s, str):
        return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

# 过滤逻辑
data_im1_filter = [
    d for d in data_im1
    if (d.get('execution_best_solution') is None or is_numeric_str(d.get('execution_best_solution'))) 
    and (
        "<thinking>" in d.get('en_math_model_coptpy_code', '') or 
        "<thinking>" in d.get('completion', '')
    )
]

data_im2_filter = [
    d for d in data_im2
    if (d.get('execution_best_solution') is None or is_numeric_str(d.get('execution_best_solution'))) 
    and (
        "<thinking>" in d.get('en_math_model_coptpy_code', '') or 
        "<thinking>" in d.get('completion', '')
    )
]

In [35]:
data_im2[0].keys()

dict_keys(['prompt', 'completion'])

In [36]:
len(data_im1_filter), len(data_im2_filter)

(1216, 14848)

In [42]:
data_im1_filter[0]

{'en_question': 'An event planning company is organizing workshops and needs to hire two types of professionals: Speakers and Coordinators. Speakers can handle 3 workshops per day and cost $500 per day, while Coordinators can manage 2 workshops per day and cost $350 per day.\n\nThe company needs to arrange professionals to handle at least 50 workshops per day, and the total cost of hiring these professionals per day cannot exceed $10,000. \n\nAdditionally, the company wants:\n- To minimize the environmental impact by reducing the commute of the professionals. Each Speaker adds 150 kg CO2 per day due to travelling, and each Coordinator adds 100 kg CO2 per day.\n- To ensure the satisfaction level of workshops, which involves maximizing the number of professionals engaged (as the diversity of professionals is thought to enrich the workshop experience).\n\nDue to event management protocols, the number of Speakers cannot exceed twice the number of Coordinators, and the number of Coordinator

In [37]:
def deduplicate_data(data_list):
    """
    根据 'en_question' 或 'question' 键对列表进行去重
    """
    seen_questions = set()
    unique_data = []
    
    for item in data_list:
        # 优先获取 'en_question'，如果没有则获取 'question'
        # 如果两个键都没有，则跳过或根据需求处理
        q_text = item.get('en_question') or item.get('question')
        d['en_question'] = q_text
        
        if q_text is not None:
            if q_text not in seen_questions:
                unique_data.append(item)
                seen_questions.add(q_text)
        else:
            # 如果数据中完全没有这两个 key，默认保留该条数据或跳过
            # 这里选择保留没有这两个 key 的数据（不参与去重逻辑）
            unique_data.append(item)
            
    return unique_data


# 2. 执行去重
data_im1_unique = deduplicate_data(data_im1_filter)
data_im2_unique = deduplicate_data(data_im2_filter)

# 如果你希望将 data_im1 和 data_im2 合并在一起进行全局去重：
# data_all_unique = deduplicate_data(data_im1 + data_im2)

print(f"data_im1 去重前: {len(data_im1)}, 去重后: {len(data_im1_unique)}")
print(f"data_im2 去重前: {len(data_im2)}, 去重后: {len(data_im2_unique)}")

data_im1 去重前: 2118, 去重后: 1153
data_im2 去重前: 15105, 去重后: 13885


In [63]:
def question2prompt(question_text: str) -> str:
    template = r"""
        You are tasked with solving an operations research problem through a structured three-step process:  
        1. Reasoning: Understand and analyze the problem.  
        2. Modeling: Formulate a precise mathematical model.  
        3. Implementation: Translate the model into executable Python code using `coptpy`.

        Please respond in the following exact format:

        <thinking>
        Your step-by-step reasoning and interpretation of the problem here.
        </thinking>
        <model>
        Your precise mathematical model formulation here, including decision variables, objective function, and constraints.
        </model>
        <code>
        Your complete and executable Python code using `coptpy` that implements the above model.
        </code>

        Question: {Question}
        """
    prompt = template.replace("{Question}", question_text.strip()).strip()
    
    return prompt

for d in data_im1_unique:

    d['prompt'] = question2prompt(d['en_question'])
    d['completion'] = d['en_math_model_coptpy_code']


for d in data_im2_unique:
    if 'prompt' not in d:
        d['prompt'] = question2prompt(d['en_question'])
        d['completion'] = d['en_math_model_coptpy_code']


In [64]:
count = 0
for d in data_im2_unique:
    if 'prompt' not in d or 'completion' not in d:
        count += 1
count

0

In [65]:
import random
# 使用 sample 保证不修改原列表顺序，同时实现随机
half_size = len(data_im1_unique) // 2
im1_selected = random.sample(data_im1_unique, half_size)
# 通过 ID 或对象对比获取剩余部分 (假设数据是可哈希的，或者直接用 index 处理)
# 这里采用打乱切片法最稳妥：
temp_list1 = list(data_im1_unique)
random.shuffle(temp_list1)
im1_selected = temp_list1[:half_size]
im1_remaining = temp_list1[half_size:]

# 处理第二个数据集 (抽 3000 条)
num_to_sample = min(3000, len(data_im2_unique))
temp_list2 = list(data_im2_unique)
random.shuffle(temp_list2)
im2_selected = temp_list2[:num_to_sample]
im2_remaining = temp_list2[num_to_sample:]


def save_json(data, filename):
    """保存数据为 JSON 文件"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"成功保存: {filename} (共 {len(data)} 条)")
    
# --- 3. 合并抽样数据 ---
combined_selected = im1_selected + im2_selected

# --- 4. 存储文件 ---

# 1. 抽样合并后的总集
save_json(combined_selected, '/home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/combined_selected.json')

# 2. 第一个数据集没被抽到的部分
save_json(im1_remaining, '/home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/im1_not_selected.json')

# 3. 第二个数据集没被抽到的部分
save_json(im2_remaining, '/home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/im2_not_selected.json')

print("-" * 30)
print("所有任务处理完成！")

成功保存: /home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/combined_selected.json (共 3576 条)
成功保存: /home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/im1_not_selected.json (共 577 条)
成功保存: /home/work/mllm_datas/yilin/code/OR-SR1/sft_data/dataset/im2_not_selected.json (共 10885 条)
------------------------------
所有任务处理完成！


In [34]:
import json

with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im1.json', 'w', encoding='utf-8') as f:
    json.dump(data_im1_unique, f, ensure_ascii=False)

with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im2.json', 'w', encoding='utf-8') as f:
    json.dump(data_im2_sample, f, ensure_ascii=False)

In [35]:
import json
import math

# 假设 data_im2_sample 是你的原始列表
total_size = len(data_im2_sample)
chunk_size = math.ceil(total_size / 3)  # 计算每份的大小

base_path = '/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im2'

for i in range(3):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    chunk = data_im2_sample[start:end]
    
    # 生成文件名，如 data_im2_part1.json
    file_path = f"{base_path}_part{i+1}.json"
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(chunk, f, ensure_ascii=False, indent=4)

print(f"拆分完成，每份约 {len(chunk)} 条数据。")

拆分完成，每份约 1332 条数据。


In [ ]:
data_1 = 